# Filling in the Gaps

A few questions were left somewhat unanswered in the lesson:

* Why does `print(bytes([10, 100, 200])) -> b'\nd\xc8'` looks so odd? 
    * What's with the `\n`? 
    * What's with the `\x`? 
    * Are the `c` and the `8` related to the `\x`?
    * Is `d` related to either `\n`, `\x`, or neither?
* How should we interpret these exotic byte literals when you encounter one _in the wild_?
* How does the magical `int.from_bytes` function work?
* How does its inverse function `int.to_bytes` work?

To answer these questions we will make a `bites` class that acts just like `bytes`, but with a few minor differences: 
* Prints itself with strings instead of the special `bytes` literals and spaces (`"00 ff"`) instead of `\x` separators.
* Will duplicate the functionality of the magical `int.from_bytes` and `int.to_bytes` methods by implementing its `bites.from_int` and `bites.to_int` methods.

Here's a skeleton of the class:

In [ ]:
class bites:
    
    def __init__(self, values):
        self.values = values
        
    def __repr__(self):
        pass
    
    def to_int(self, byte_order):
        pass
    
    @classmethod
    def from_int(cls, n, length, byte_order):
        pass    

# Implementing `bites.from_int` in 5 Steps

A "byte" is a value `x` such that `0 <= x < 256`.

Simple.

How can we represent a number 256 or higher as a series of bytes? Hmm ...

Let's think about how we solve this with our normal decimal numbers. Instead of 256 separate values or "places", we only have 10 places with values `y` such that `0 <= y < 9`.

What happens when we encounter a number outside this range? Say the number after 9 -- `9 + 1`? We no longer have a "numberal" or number symbol to represent this number. We could invent one of course, but quickly becomes unworkable due to the memorization demands.

Instead we employ an ingenius trick to utilize _two numberals to rempresent 1 number_. It's really clever when you think of it. Of course, we write this `9 + 1` as `10` -- the left character represents that we have run out of numberals 1 time. It's the 10's place. The right numberal says that we haven't started going up again. We can repeat this procedure indefinitely

```

  ----------  ...  ---------- ---------- ----------
    10**n's          100's       10's       1's  

```

Therefore, any number `N` can be represented as a sum of 

```
an *  10**n + ... + a2 *  10**2 + a1 *  10**1 + a0 *  10**0
```

And can employ exactly the same trick to represent numbers greater than 256 as a series of bytes -- numbers 0 <= x < 256. Just replaces the old threshold 10 with the new threshold 256:

```
an * 256**n + ... + a2 * 256**2 + a1 * 256**1 + a0 * 256**0
```

The trick here is to go from an `int` to a list of values which each represent one place in the base 256 expansion of the `int` we're dealing with.

So we have a representation. We could use a `simple` data structure to store this: `[an, ..., a2, a1, a0]`. With say a0 = 0, a1 = 1, and a2 = 2 we would have `[2, 1, 0]`. This would save the number `2 * 256**2 + 1 * 256**1 + 0 * 256**0 = 65792` as 3 bytes. Simple enough!

Now we just need an algorithm to go back and forth between normal Python `int`s. I believe the "int -> base 256 places" case is more intuitive because it's simple multiplication:

In [ ]:
N = 92837365

print("binary: ", bin(N))
print("octal: ", oct(N))
print("hexidecimal: ", hex(N))

Here is some logic to decompose `92837365` into base 256 places:

In [ ]:
places = []
n = N  # let's not modify N

while n > 0:
    places.insert(0, n % 256)
    n = n // 256

print(places)

## Step 1:  `int_to_256_places(n)`

Next, turn this into a function `int_to_256_places(n)`

In [ ]:
def int_to_base_256_places(n):
    places = []
    while n > 0:
        places.insert(0, n % 256)
        n = n // 256
    return places

In [ ]:
def test():
    assert int_to_base_256_places(N) == [5, 136, 149, 245]
    
test()

## Step 2: `int_to_places(n, base)`

Now add a second parameter to the function and rename it to `int_to_places(n, base)` so that it works with any base.

Test it against the binary, octal, and hex representations we say above. For example, we saw `hex(92837365)` was `0x58895f5`, so as a list it should be `[5, 8, 8, 9, 5, 15, 5]` (hex `f` equal to decimal `15`).

In [ ]:
def int_to_places(n, base):
    pass

def int_to_places(n, base):
    places = []
    while n > 0:
        places.insert(0, n % base)
        n = n // base
    return places

print(int_to_places(N, 16))  # it this what you expect?

In [ ]:
# Some automated tests to make absolutely sure you're correct!

def test():
    assert int_to_places(N, 256) == [5, 136, 149, 245]
    assert int_to_places(N, 16) == [int(x, 16) for x in hex(N)[2:]]
    assert int_to_places(N, 2) == [int(x, 2) for x in bin(N)[2:]]
    
test()

## Step 3: `int_to_places(n, base, length)`

Remember how every field in the protocol docs' "Message Structure" table has a `length` attribute?

![image](../images/message-structure.png)

We need to be able to support that, too. We should be able to say `int_to_places(1, 4)` and get [0, 0, 0, 1]. This feature helps us interpret and produce n-byte integer fields we cencounter in the Bitcoin protocol.

In [ ]:
def int_to_places(n, base, length):        
    places = []
    while len(places) < length:
        places.insert(0, n % base)
        n = n // base
    # FIXME: keep this???
#     if n != 0:
#         raise ValueError(f'"{n}" cannot be expressed in "{length}" bites')
    return places

def test():
    assert int_to_places(N, 256, 10) == [0] * 6 + [5, 136, 149, 245]
    
    vals = [int(x, 16) for x in hex(N)[2:]]
    zeros = [0] * (20 - len(vals))
    assert int_to_places(N, 16, 20) == zeros + vals

test()

## Step 4:  `int_to_places(n, base, length, byte_order)`

This is how we've been choosing to represent the number `92837365`: `[5, 136, 149, 245]`

The larger places on the left, smaller places to the right. Just like familiar decimal nubmers

```
    5     136     149     245
-----   -----   -----   -----
264^3   264^2   264^1   264^0
```

But isn't this choice completely arbitrary? Why not do the opposite: smaller places to the left, larger places to the right? 

```
  245     149     136       5
-----   -----   -----   -----
264^0   264^1   264^2   264^3
```

There are names for these:
* Big Endian
* Little Endian

Somewhat confusingly, different areas of computer science prefer one way or the other: computers generally use "little endian" to store information internally, and "big endian" for network transmission. Even within Bitcoin, Satoshi didn't choose one way or the other. Satoshi generally preferred "little endian" but he encodes IP addresses using "big endian", for instance. What a mess!

You can find an interesting discussion of "endianness" [here](https://bitcoin.stackexchange.com/questions/2063/why-does-the-bitcoin-protocol-use-the-little-endian-notation) and a nice YouTube video [here](https://www.youtube.com/watch?v=seZLUbgbB7Y)

What we've been doing is "Bit Endian" byte order. Add another parameter to our function so that it can handle "Little Endian" byte order. If `byte_order` is `"little"` you just need to reverse the list:

In [ ]:
def int_to_places(n, base, length, byte_order):
    places = []
    while len(places) < length:
        places.insert(0, n % base)
        n = n // base
    if byte_order == "little":
        places.reverse()
    return places

In [ ]:
def test():
    vals = [5, 136, 149, 245]
    assert int_to_places(N, 256, len(vals), 'big') == vals
    assert int_to_places(N, 256, len(vals), 'little') == vals[::-1]

    vals = [int(x, 16) for x in hex(N)[2:]]
    assert int_to_places(N, 16, len(vals), 'big') == vals
    assert int_to_places(N, 16, len(vals), 'little') == vals[::-1]

test()

## Step 5:  `bites.from_int(n, base, length, byte_order)`


Let's put it all together. Fill out the `from_int` method below and get the tests to pass

In [ ]:
class bites:
    
    def __init__(self, values):
        self.values = values
        
    def __repr__(self):
        pass
    
    def to_int(self, num_bytes, byte_order):
        pass
    
    @classmethod
    def from_int(cls, n, length, byte_order):
        places = int_to_places(n, 256, length, byte_order)
        return cls(places)

In [ ]:
def test():
    vals = [5, 136, 149, 245]
    assert bites.from_int(N, len(vals), 'big').values == vals
    assert bites.from_int(N, len(vals), 'little').values == vals[::-1]
    
test()

# Implement `bites.to_int(places)` in FIXME steps

## Step 1: `base_256_places_to_int(places)`

Write inverse function to `int_to_base_256_places` such that:

`base_256_places_to_int([5, 136, 149, 245]) -> 92837365`

In [ ]:
def base_256_places_to_int(places):
    n = 0
    while len(places):
        place = places.pop(0)
        n += "FIXME"
    return n

def base_256_places_to_int(places):
    n = 0
    for index, place in enumerate(reversed(places)):
        n += place * 256 ** index
    return n

In [ ]:
def test():
    assert base_256_places_to_int([5, 136, 149, 245]) == N

test()

## Step 2: `places_to_int(places, base)`

Modify `base_256_places_to_int` so that it accepts arbitrary bases:

In [ ]:
def places_to_int(places, base):
    "FIXME"
    
def places_to_int(places, base):
    n = 0
    for index, place in enumerate(reversed(places)):
        n += place * base ** index
    return n

In [ ]:
def test():
    assert places_to_int([5, 136, 149, 245], 256) == N
    assert places_to_int([int(x, 16) for x in hex(N)[2:]], 16) == N
    assert places_to_int([int(x, 2) for x in bin(N)[2:]], 2) == N
test()

## Step 3: `places_to_int(places, base, byte_order)`

Add byte order

In [ ]:
def places_to_int(places, base, byte_order):
    "FIXME"

def places_to_int(places, base, byte_order):
    if byte_order == 'big':
        places = reversed(places)
    n = 0
    for index, place in enumerate(places):
        n += place * base ** index
    return n

In [ ]:
def test():
    assert places_to_int([5, 136, 149, 245], 256, 'big') == N
    assert places_to_int([5, 136, 149, 245][::-1], 256, 'little') == N
    
    assert places_to_int([int(x, 16) for x in hex(N)[2:]], 16, 'big') == N
    assert places_to_int([int(x, 16) for x in hex(N)[2:]][::-1], 16, 'little') == N

test()

## Step 4: `bites.to_int(base, byte_order)`

Bring it all together

In [ ]:
class bites:
    
    def __init__(self, values):
        self.values = values
        
    def __repr__(self):
        return repr(self.values)
    
    def to_int(self, byte_order):
        return places_to_int(self.values, 256, byte_order)
    
    @classmethod
    def from_int(cls, n, length, byte_order):
        places = int_to_places(n, 256, length, byte_order)
        return cls(places)
    
    @classmethod
    def from_hex(cls, hx):
        # FIXME
        places = [int(hx[i:i+2], 16) for i in range(0, len(hx), 2)]
        return cls(places)
    
    # FIXME
    def __eq__(self, other):
        return self.values == other.values

In [ ]:
def test():
    places = [5, 136, 149, 245]
    assert bites(places).to_int('big') == N
    assert bites(places[::-1]).to_int('little') == N

test()

# `bites.__repr__`

The representations of Python objects are determined by `.__repr__()` methods.

Let's see `bytes.__repr__` in action:

In [ ]:
for i in range(256):
    print(i, "->", bytes([i]))

I want you to implement a function that can print `bites` instances in the same way. To assist with this one I'm going to give you a list of character codes that have special meaning to `bytes`.

Below is a dictionary containing an `int -> ascii character` mapping of all numbers in 0 <= x < 256 with special meaning to `bytes`.

In [310]:
from utils import special_chars

print(special_chars)

{9: '\t', 10: '\n', 13: '\r', 32: ' ', 33: '!', 34: '"', 35: '#', 36: '$', 37: '%', 38: '&', 39: "'", 40: '(', 41: ')', 42: '*', 43: '+', 44: ',', 45: '-', 46: '.', 47: '/', 48: '0', 49: '1', 50: '2', 51: '3', 52: '4', 53: '5', 54: '6', 55: '7', 56: '8', 57: '9', 58: ':', 59: ';', 60: '<', 61: '=', 62: '>', 63: '?', 64: '@', 65: 'A', 66: 'B', 67: 'C', 68: 'D', 69: 'E', 70: 'F', 71: 'G', 72: 'H', 73: 'I', 74: 'J', 75: 'K', 76: 'L', 77: 'M', 78: 'N', 79: 'O', 80: 'P', 81: 'Q', 82: 'R', 83: 'S', 84: 'T', 85: 'U', 86: 'V', 87: 'W', 88: 'X', 89: 'Y', 90: 'Z', 91: '[', 92: '\\', 93: ']', 94: '^', 95: '_', 96: '`', 97: 'a', 98: 'b', 99: 'c', 100: 'd', 101: 'e', 102: 'f', 103: 'g', 104: 'h', 105: 'i', 106: 'j', 107: 'k', 108: 'l', 109: 'm', 110: 'n', 111: 'o', 112: 'p', 113: 'q', 114: 'r', 115: 's', 116: 't', 117: 'u', 118: 'v', 119: 'w', 120: 'x', 121: 'y', 122: 'z', 123: '{', 124: '|', 125: '}', 126: '~'}


Anything value left unassigned by that dictionary should be converted into it's hexidecimal representation -- using hex represeents a comprimise because it doesn't require memorizing 256 different characters to be able to represent all of these numbers in 1 character. 

But it's more space efficient than decimal for representing bytes -- hex can represent every byte in 2 characters while decimal would need 3 characters for 60% of all byte values (everything over 99).

### Exercise: Implement a  `represent` function that works exactly like `bytes.__repr__` does

Here's how it should work:

`represent(bytes([145, 22, 75, 152, 83])) -> "\x91\x16K\x98S"`

The output should be a string. Hint: to put a `\` in a string you need to escape like `\\`. So a newline `\n` for example would need to be input as `\\n`.

In [316]:
def represent(b):
    # let's operate on the underlying numbers
    numbers = list(b)
    result = ""
    for n in numbers:
        if n in special_chars:
            result += special_chars[n]
        else:
            result += '\\x' + hex(n)[2:]
    return result
            
print(represent(bytes([145, 22, 75, 152, 83])))

\x91\x16K\x98S


In [317]:
def test():
    assert represent(bytes([145, 22, 75, 152, 83])) == "\\x91\\x16K\\x98S"

test()

# Put it all together (with some help from our friends)

I'm going to add 2 methods that our Lesson 1 code requires: `.strip` and `__eq__`. To simplify things I just convert to `bytes` and have it do all the work ...

In [338]:
class bites:
    
    def __init__(self, values):
        self.values = values
    
    def __eq__(self, other):
        if isinstance(other, bytes):
            return self.values == list(other)
        return self.values == other.values
    
    def __repr__(self):
        result = ""
        for n in self.values:
            if n in special_chars:
                result += special_chars[n]
            else:
                result += '\\x' + hex(n)[2:]
        return result
    
    def to_int(self, byte_order):
        return places_to_int(self.values, 256, byte_order)
    
    @classmethod
    def from_int(cls, n, length, byte_order):
        places = int_to_places(n, 256, length, byte_order)
        return cls(places)

    def strip(self, pattern):
        return bites(list(bytes(self.values).strip(pattern)))

In [ ]:
# __eq__ needed for  magic bytes comparisons

b'\xf9\xbe\xb4\xd9' == bites([0xF9, 0xBE, 0xB4, 0xD9])

In [318]:
# strip() needed for reading commands

b = bites(list(b"version\x00\x00\x00\x00\x00"))

print("unstripped:", b)
print("stripped:", b.strip(b"\x00"))

unstripped: version\x00\x00\x00\x00\x00
stripped: version


### BiteStream

This class turns `bytes`treams into `bites`treams

In [ ]:
class BitesStream:

    def __init__(self, stream):
        self.stream = stream

    def read(self, n):
        # return bites.from_hex(self.stream.read(n).hex())
        return bites(list(self.stream.read(n)))
        
    def __getattr__(self, name):
        return getattr(self.f, name)

### Hashing `bites`

`hashlib.sha256` requires inputs to implement the "Buffer API".This 

In [ ]:
from hashlib import sha256

def hash256(b):
    values = sha256(sha256(bytes(b.values)).digest()).digest()
    return b.__class__(values)

# Reading Bitcoin Messages From `bites`

A couple small tweeks to make our `NetworkEnvelope` class developed in Lesson 1 work with `bites` instead of `bytes`


### hex reader

This helper class will allow us to read hex strings out of our 

In [323]:
class BitesStream:

    def __init__(self, stream):
        self.stream = stream

    def read(self, n):
        # return bites.from_hex(self.stream.read(n).hex())
        return bites(list(self.stream.read(n)))
        
    def __getattr__(self, name):
        return getattr(self.f, name)

### Update the hash function to work with `bites`

In [345]:
from hashlib import sha256

def calculate_checksum(b):
    values = sha256(sha256(bytes(b.values)).digest()).digest()
    checksum_values = values[:4]
    return b.__class__(list(checksum_values))

### Our job is done here!

In [348]:

class NetworkEnvelope:

    def __init__(self, command, payload):
        self.command = command
        self.payload = payload

    @classmethod
    def from_stream(cls, stream):
        magic = stream.read(4)
        if magic != NETWORK_MAGIC:
            raise ValueError('Network magic is wrong')

        command = stream.read(12).strip(b"\x00")
        payload_length = stream.read(4).to_int('little')
        checksum = stream.read(4)
        payload = stream.read(payload_length)
        
        if checksum != calculate_checksum(payload):
            raise RuntimeError("Checksums don't match")

        return cls(command, payload)

    def serialize(self):
        raise NotImplementedError()

    def __repr__(self):
        return f"<Message command={self.command}>"


In [349]:
import socket

# magic "version" bytestring
VERSION = b'\xf9\xbe\xb4\xd9version\x00\x00\x00\x00\x00j\x00\x00\x00\x9b"\x8b\x9e\x7f\x11\x01\x00\x0f\x04\x00\x00\x00\x00\x00\x00\x93AU[\x00\x00\x00\x00\x0f\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0f\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00rV\xc5C\x9b:\xea\x89\x14/some-cool-software/\x01\x00\x00\x00\x01'

PEER_IP = "67.164.73.145"
PEER_PORT = 8333

sock = socket.socket()
sock.connect((PEER_IP, PEER_PORT))
stream = sock.makefile('rb')
bites_stream = BitesStream(stream)

# initiate the "version handshake"
sock.send(VERSION)

# receive their "version" response
msg = NetworkEnvelope.from_stream(bites_stream)

print(msg)
print(msg.payload)

<Message command=version>
\x4\x0\x0\x0\x0\x0\x0\x0\x0\x0\x0\x0\x0\x0\x0\x0\x0\x0\x0\x0\x0\x0\x0\x0\x0\x9d\xaf:s\x8f\xfdZ)\x10/Satoshi:0.17.0/\xa3\x8d\x8\x0\x1
